### chatcompletition, Prompt Template, LLM api call with Open Source LLMs,with local storage

In [1]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)  

Index.fit(documents)

In [2]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [3]:
def build_prompt(query, search_results):
    prompt_template = """ You are an expert in machine learning and MLOps, assisting a junior engineer. Your task is to answer the following question using only the provided context from the FAQ database. Do not include any information that is not present in the context. If the context does not provide an answer to the question, respond with "Not FOUND in the context given" and provide a brief explanation.

**QUESTION:** How do I start using MLflow?

**CONTEXT:** 
- Section: Module 2: Experiment Tracking
  - Question: Viewing MLflow Experiments using MLflow CLI
    - Answer: Problem: After starting the tracking server, when we try to use the mlflow CLI commands, most of them can’t find the experiments that have been run with the tracking server. Solution: Set the environment variable `MLFLOW_TRACKING_URI` to the URI of the SQLite database (e.g., `export MLFLOW_TRACKING_URI=sqlite:///{path to sqlite database}`). After this, view experiments from the command line using commands like `mlflow experiments search`.

  - Question: When using Autologging, do I need to set a training parameter to track it on MLflow UI?
    - Answer: No, autologging tracks the parameters automatically, even if you do not explicitly set them when calling `.fit`. You can set only the parameters you want during training, and all parameters will be visible in the MLflow UI.

  - Question: WARNING: mlflow.sklearn: Failed to log training dataset information to MLflow Tracking.
    - Answer: This warning occurs because MLflow expects the dataset to be in a `pd.DataFrame` format, but if you provide a `numpy.ndarray`, it fails. To avoid this, set `log_datasets = False` in the autolog function.

  - Question: mlflow not showing artifacts
    - Answer: When using local storage, start MLflow where `mlflow.db` is located. For example, if `mlflow.db` is in `mlops/mlflow`, navigate to that folder and run `../scripts/start.sh`.

**ANSWER:**
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [4]:
Query = "How to use mlflow for experiment tracking?"
def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

### Mistral-7B openSource LLM

In [5]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline

In [6]:
from huggingface_hub import login

In [7]:
login(os.getenv("HF_zoomcamp"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /teamspace/studios/this_studio/.cache/huggingface/token
Login successful


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.3", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3", padding_side="left")


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
generator = pipeline("text-generation", model = model, tokenizer = tokenizer)

In [10]:
prompt = """ You are an expert in machine learning and MLOps, assisting a junior engineer. Your task is to answer the following question using only the provided context from the FAQ database. Do not include any information that is not present in the context. If the context does not provide an answer to the question, respond with "Not FOUND in the context given" and provide a brief explanation.

**QUESTION:** How do I start using MLflow?

**CONTEXT:** 
- Section: Module 2: Experiment Tracking
  - Question: Viewing MLflow Experiments using MLflow CLI
    - Answer: Problem: After starting the tracking server, when we try to use the mlflow CLI commands, most of them can’t find the experiments that have been run with the tracking server. Solution: Set the environment variable `MLFLOW_TRACKING_URI` to the URI of the SQLite database (e.g., `export MLFLOW_TRACKING_URI=sqlite:///{path to sqlite database}`). After this, view experiments from the command line using commands like `mlflow experiments search`.

  - Question: When using Autologging, do I need to set a training parameter to track it on MLflow UI?
    - Answer: No, autologging tracks the parameters automatically, even if you do not explicitly set them when calling `.fit`. You can set only the parameters you want during training, and all parameters will be visible in the MLflow UI.

  - Question: WARNING: mlflow.sklearn: Failed to log training dataset information to MLflow Tracking.
    - Answer: This warning occurs because MLflow expects the dataset to be in a `pd.DataFrame` format, but if you provide a `numpy.ndarray`, it fails. To avoid this, set `log_datasets = False` in the autolog function.

  - Question: mlflow not showing artifacts
    - Answer: When using local storage, start MLflow where `mlflow.db` is located. For example, if `mlflow.db` is in `mlops/mlflow`, navigate to that folder and run `../scripts/start.sh`.

**ANSWER:**
""".strip()

In [14]:
response = generator(prompt, max_length=1024, do_sample=True, temperature=0.7,top_p = 0.95, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
print(response[0]['generated_text'])

You are an expert in machine learning and MLOps, assisting a junior engineer. Your task is to answer the following question using only the provided context from the FAQ database. Do not include any information that is not present in the context. If the context does not provide an answer to the question, respond with "Not FOUND in the context given" and provide a brief explanation.

**QUESTION:** How do I start using MLflow?

**CONTEXT:** 
- Section: Module 2: Experiment Tracking
  - Question: Viewing MLflow Experiments using MLflow CLI
    - Answer: Problem: After starting the tracking server, when we try to use the mlflow CLI commands, most of them can’t find the experiments that have been run with the tracking server. Solution: Set the environment variable `MLFLOW_TRACKING_URI` to the URI of the SQLite database (e.g., `export MLFLOW_TRACKING_URI=sqlite:///{path to sqlite database}`). After this, view experiments from the command line using commands like `mlflow experiments search`.



In [30]:
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'A list of colors: red, blue, green, yellow, orange, purple, pink,'

In [31]:
def llm_call(prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs,max_length=1500)
    result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return result

In [32]:
rag("How do i start using mlflow?")

KeyError: 'path to sqlite database'

In [ ]:
print(_)